In [1]:
####################################
#ENVIRONMENT SETUP

In [2]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py
from tqdm import tqdm

In [3]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [4]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [5]:
#IMPORT FUNCTIONS
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
import FUNCTIONS_Variable_Calculation
from FUNCTIONS_Variable_Calculation import *

In [6]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=2)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="Tracking_Algorithms", dataName="Lagrangian_UpdraftTracking",
                                dtype='float32',codeSection = "Project_Algorithms")

=== CM1 Data Summary ===
 Simulation #:   2
 Resolution:     1km
 Time step:      1min
 Vertical levels:95
 Parcels:        50e6
 Data file:      /mnt/lustre/koa/scratch/air673/cm1out_1km_1min_95nz.nc
 Parcel file:    /mnt/lustre/koa/scratch/air673/cm1out_pdata_1km_1min_50e6np.nc
 Time steps:     661

=== DataManager Summary ===
 inputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData
 outputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Project_Algorithms/Tracking_Algorithms
 inputDataDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData/1km_1min_95nz/ModelData
 inputParcelDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData/1km_1min_95nz/ParcelData
 outputDataDirectory #:   /mnt

In [7]:
#data manager class (for saving data)
DataManager_TrackedProfiles = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="Tracked_Profiles", dataName="Tracked_Profiles",
                                dtype='float32',codeSection = "Project_Algorithms")

=== DataManager Summary ===
 inputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData
 outputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Project_Algorithms/Tracked_Profiles
 inputDataDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData/1km_1min_95nz/ModelData
 inputParcelDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData/1km_1min_95nz/ParcelData
 outputDataDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Project_Algorithms/Tracked_Profiles/1km_1min_95nz/Tracked_Profiles



In [8]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"3_Project_Algorithms","2_Tracking_Algorithms"))
from CLASSES_TrackingAlgorithms import TrackingAlgorithms_DataLoading_Class, Results_InputOutput_Class, TrackedParcel_Loading_Class

In [9]:
# IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"3_Project_Algorithms","3_Tracked_Profiles"))
from CLASSES_TrackedProfiles import TrackedProfiles_DataLoading_CLASS

In [10]:
import sys
path=os.path.join(mainCodeDirectory,'Functions/')
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

#####

#Import StatisticalFunctions 
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import StatisticalFunctions
from StatisticalFunctions import * # import NumericalFunctions 

In [11]:
##############################################
#JOB ARRAY

In [12]:
#JOB ARRAY SETUP
UsingJobArray=True

def GetNumJobs(res,t_res):
    if res=='1km':
        if t_res=='5min':
            num_jobs=132
        elif t_res=='1min':
            num_jobs=660
    elif res=='250m': 
        if t_res=='1min':
            num_jobs=660
    return num_jobs
num_jobs = GetNumJobs(ModelData.res,ModelData.t_res)
SlurmJobArray = SlurmJobArray_Class(total_elements=ModelData.Ntime, num_jobs=num_jobs, UsingJobArray=UsingJobArray)
start_job = SlurmJobArray.start_job; end_job = SlurmJobArray.end_job

def GetNumElements():
    loop_elements = np.arange(ModelData.Ntime)[start_job:end_job]
    return loop_elements
loop_elements = GetNumElements()

Running timesteps from 0:1 



In [545]:
##############################################
#DATA LOADING FUNCTIONS

In [546]:
def MakeDataDictionary(variableNames,t,printstatement=False):
    timeString = ModelData.timeStrings[t]
    # print(f"Getting data from {timeString}","\n")
    
    dataDictionary = {variableName: CallLagrangianArray(ModelData, DataManager, timeString, variableName=variableName, printstatement=printstatement) 
                      for variableName in variableNames}      
    return dataDictionary
    
def GetSpatialData(t):    
    variableNames = ['Z','Y','X']
    dataDictionary = MakeDataDictionary(variableNames,t)
    [Z,Y,X] = (dataDictionary[k] for k in variableNames)
    return Z,Y,X

def GetLangrangianBinaryArray(t):
    variableNames=['PROCESSED_A_g','PROCESSED_A_c']
    binaryDictionary = MakeDataDictionary(variableNames,t)
    
    A_g = binaryDictionary['PROCESSED_A_g']
    A_c = binaryDictionary['PROCESSED_A_c']

    return A_g,A_c

In [547]:
########################################
#RUNNING FUNCTIONS

In [548]:
#Functions for Initializing Profile Arrays
def CopyStructure(dictionary, placeholder=None):
    """Deep-copy dictionary structure, replacing leaves with a given placeholder."""
    if isinstance(dictionary, dict):
        return {k: CopyStructure(v, placeholder) for k, v in dictionary.items()}
    else:
        return placeholder
        
def InitializeHistograms(trackedArrays, varNames, time_bins,zBins_km_g,zBins_km_c, property_bins_Dictionary):
    """
    Create a nested structure matching trackedArrays,
    with empty histogram arrays for each variable:
        - var_hist2d
        - var_parcel_last_time_hist2d
    """
    
    histogramsDictionary = {}
    n_time = len(time_bins) - 1
    
    for category, depth_dict in trackedArrays.items():  # e.g. 'CL', 'SBF'
        histogramsDictionary[category] = {}

        for depth_type in depth_dict.keys():  # e.g. 'ALL', 'SHALLOW', 'DEEP'
            histogramsDictionary[category][depth_type] = {}

            for varName in varNames:

                # ---- initialize varName level
                histogramsDictionary[category][depth_type][varName] = {}
                for mode in ["g","c"]:
                    histogramsDictionary[category][depth_type][varName][mode] = {}

                # number of property bins for this variable
                n_prop = len(property_bins_Dictionary[varName]) - 1

                # initialized z-subsetted empty histograms
                for mode, zBins in {
                    "g": zBins_km_g,
                    "c": zBins_km_c,
                }.items():
                    for z1, z2 in zBins:
                        zKey = f"{z1}_{z2}km_hist2d"
                        histogramsDictionary[category][depth_type][varName][mode][zKey] = np.zeros(
                            (n_time, n_prop)
                        )

    return histogramsDictionary

In [549]:
def GetParcelNumbers(trackedArray, t):
    """
    Return all parcel indices (p) and their corresponding row indices
    for parcels that are active at time t.
    Vectorized, no row-by-row loops.
    """
    t_start = trackedArray[:, 1]
    t_end   = np.minimum(trackedArray[:, 2] + trackedArray[:, 3], ModelData.Ntime)

    # Boolean mask for rows active at time t
    mask = (t >= t_start) & (t <= t_end)

    # Extract parcel numbers and their corresponding row indices
    selectedRows = np.where(mask)[0]
    selectedPs = trackedArray[selectedRows, 0]
    leftRightIndexes = trackedArray[selectedRows, 4]

    return selectedRows, selectedPs, leftRightIndexes

In [550]:
#FUNCTIONS FOR GETTING GRID BOX MATCHES

def GetGridBoxMatches_V1(Z,Y,X, zLevels,yLevels,xLevels):
    gridboxMatches = [
        np.where((Z == zLevel) & (Y == yLevel) & (X == xLevel))[0]
        for zLevel, yLevel, xLevel in zip(zLevels, yLevels, xLevels)
    ]
    if len(gridboxMatches) == 0:
        return None
    return gridboxMatches

from collections import defaultdict
def BuildGridboxIndex(Z, Y, X):
    gridIndex = defaultdict(list)
    for i in range(len(Z)):
        gridIndex[(Z[i], Y[i], X[i])].append(i)
    return gridIndex
def GetGridBoxMatches_V2(Z,Y,X, zLevels,yLevels,xLevels):
    gridIndex = BuildGridboxIndex(Z, Y, X)
    gridboxMatches = [
        np.asarray(gridIndex[(z, y, x)], dtype=int)
        for z, y, x in zip(zLevels, yLevels, xLevels)
    ]
    if len(gridboxMatches) == 0:
        return None
    return gridboxMatches


# def CheckIfSame_GridBoxMatches(one,two):
#     same = (
#         len(one) == len(two)
#         and all(
#             np.array_equal(a, b)
#             for a, b in zip(one, two)
#         )
#     )
#     print(same,"#"*10,"\n")

# gridboxMatches_original = GetGridBoxMatches_V1(Z,Y,X, zLevels,yLevels,xLevels)
# gridboxMatches = GetGridBoxMatches_V2(Z,Y,X, zLevels,yLevels,xLevels)
# CheckIfSame_GridBoxMatches(gridboxMatches_original,gridboxMatches)   

In [551]:
#FUNCTIONS FOR APPLYING CLOUD MASK TO PARCELS

def GetEntrainmentMask(A_g,A_g_Prior,
                       A_c,A_c_Prior,
                       selectedPs):
    mask_g = (A_g & (~A_g_Prior)).astype(bool)
    mask_g[selectedPs] = False #remove the selected parcels themselves
    mask_c = (A_c & (~A_c_Prior)).astype(bool)            
    mask_c[selectedPs] = False #remove the selected parcels themselves
    return mask_g,mask_c
    
# def GetWhereOtherEntrainedParcels_V1(mask_c,gridboxMatches):
#     whereOtherEntrainedParcels_c = [idx[mask_c[idx]]
#                                     for idx in gridboxMatches]
#     if len(whereOtherEntrainedParcels_c) == 0:
#         return None
#     collapsed = np.concatenate(whereOtherEntrainedParcels_c)
#     return collapsed

def GetWhereOtherEntrainedParcels_V2(mask_c,gridboxMatches):
    collapsed = np.concatenate(gridboxMatches)
    collapsed = collapsed[mask_c[collapsed]]
    if collapsed.size == 0:
        return None
    return collapsed
    
# def CheckIfSame_WhereOtherEntrainedParcel(one,two):
#     same = np.array_equal(np.sort(one),
#                           np.sort(two))
#     print(same,"#"*10,"\n")
# collapsed_original = GetWhereOtherEntrainedParcels_V1(mask_c,gridboxMatches)
# collapsed = GetWhereOtherEntrainedParcels_V2(mask_c,gridboxMatches)
# CheckIfSame_WhereOtherEntrainedParcel(collapsed_original,collapsed)

In [552]:
#FUNCTIONS FOR MAKING PROPERTY HISTOGRAM

def AccumulatePropertyHistogram(histogramsDictionary,
                                key1,key2,varName,
                                array,Z,
                                collapsed_g,collapsed_c,
                                relative_time,
                                property_bins_Dictionary,
                                time_bins,zBins_km_g,zBins_km_c):
    for mode, collapsed, zBins_km in (
        ("g", collapsed_g, zBins_km_g),
        ("c", collapsed_c, zBins_km_c),
    ):
        if (collapsed is None): continue

        #GETTING PROPERTY HISTOGRAMS
        ##########
        # property values at this time for these entrained parcels
        properties = array[collapsed]
        zVals_km = ModelData.zh[Z[collapsed]]
        
        # time arrays for histogram2d
        times = np.full(properties.shape, relative_time)
        # property bins for each varName
        property_bins = property_bins_Dictionary[varName]
    
        # ==========================================================
        # Make histograms
        # ==========================================================
        for z1, z2 in zBins_km:
            zMask = (zVals_km >= z1) & (zVals_km < z2)
            if not np.any(zMask):
                continue
            property_hist2d_Z, _, _ = np.histogram2d(
                times[zMask],
                properties[zMask],
                bins=(time_bins, property_bins)
            )
    
            zKey = f"{z1}_{z2}km_hist2d"
            histogramsDictionary[key1][key2][varName][mode][zKey] += property_hist2d_Z

In [553]:
def MakeTrackedProfiles(trackedArrays,histogramsDictionary,property_bins_Dictionary,varNames,
                        Z,Y,X,t, A_g,A_c,A_g_Prior,A_c_Prior,
                        zBins_km_g,zBins_km_c,
                        printstatement=True):
    """
    Update profileArraysDictionary with variable data for parcels active at time t.
    Accumulates sums and counts in both profile_array and profile_array_squares.
    """
    #CALCULATING
    for key1, subdict in trackedArrays.items():         # e.g. 'CL', 'SBF'
        print("\t",f'working on {key1}')
        for key2, trackedArray in subdict.items():           # e.g. 'ALL', 'DEEP'
            print("\t\t",f'working on {key2}')
    
            #Part 1: getting parcels in trackedArray to run through
            if printstatement: print(f"Part 1: getting parcels in trackedArray to run through")
                
            _, selectedPs, leftRightIndexes = GetParcelNumbers(trackedArray, t) #get parcels that are counted at time t
            if printstatement: print(f"\tRunning for {len(selectedPs)} Parcels")
            
            #getting Z,Y,X data
            zLevels = Z[selectedPs]; yLevels = Y[selectedPs]; xLevels = X[selectedPs]

            #Part 2: find which other parcels exist in each grid box
            if printstatement: print(f"Part 2: find which other parcels exist in each grid box")
                
            # Step a: compute spatial matches once
            if printstatement: print("\tStep a: compute spatial matches once") #SLOW POINT HERE
            gridboxMatches = GetGridBoxMatches_V2(Z,Y,X, zLevels,yLevels,xLevels)
            if gridboxMatches is None:
                continue

            #Part 3: find which of those parcels were entrained into a general/cloudy updraft
            if printstatement: print(f"Part 3: find which of those parcels were entrained into a general/cloudy updraft")
            
            # Step a: compute entrainment masks
            if printstatement: print("\tStep a: compute entrainment masks")
            mask_g,mask_c = GetEntrainmentMask(A_g,A_g_Prior,
                                               A_c,A_c_Prior,
                                               selectedPs)

            # Step b: apply masks to find all parcels
            if printstatement: print("\tStep b: apply masks to find all parcels") #SLOW POINT HERE
            collapsed_g = GetWhereOtherEntrainedParcels_V2(mask_g,gridboxMatches)
            collapsed_c = GetWhereOtherEntrainedParcels_V2(mask_c,gridboxMatches)
            if (collapsed_g is None) and (collapsed_c is None): continue

            # Step c: track parcels back (last 30 minutes) and read properties
            if printstatement: print("\tStep c: track parcels back (last 60 minutes) and read properties")
            
            trackTimes = np.arange(t,(t-timesteps_per_hour)-1,-1)
            for count, t_back in enumerate(tqdm(trackTimes,desc="\t\tTracking back parcels",leave=False)):
                relative_time = t_back - t

                VARs = MakeDataDictionary(varNames, t_back)   
                for varName, array in VARs.items():
                    #GETTING PROPERTY HISTOGRAMS
                    AccumulatePropertyHistogram(histogramsDictionary,
                                                key1,key2,varName,
                                                array,Z,
                                                collapsed_g,collapsed_c,
                                                relative_time,
                                                property_bins_Dictionary,
                                                time_bins,zBins_km_g,zBins_km_c)
    return histogramsDictionary

In [554]:
########################################
#RUNNING

In [16]:
#Loading in Tracked Parcels Info
trackedArrays,LevelsDictionary = TrackedParcel_Loading_Class.LoadingSubsetParcelData(ModelData,DataManager,
                                                         Results_InputOutput_Class)
trackedArrays.pop("ColdPool") #removing this extra unneeded category

#needed parameters
timesteps_per_min = 1/(ModelData.time[1].item()/1e9/60 )
timesteps_per_hour = int(60*timesteps_per_min)
qcqi_thresh = 1e-6
# time_bins = np.arange(0,(0-timesteps_per_hour)-1,-1)[::-1]
time_bins = np.arange(0.5, -timesteps_per_hour-1.5, -1)[::-1]

#variables 
varNames = ["QV", "QCQI", "W", "THETA_v"]

#property bins for each variable
n_bins = 500
property_bins_Dictionary = {
    "QV":    np.linspace(0, 20/1e3, n_bins),        # water vapor mixing ratio
    "QCQI":  np.linspace(1e-6, 1e-3, n_bins),         # cloud+ice mixing ratio
    "W":     np.linspace(-5, 10, n_bins),         # vertical velocity bins
    "THETA_v":    np.linspace(300, 320, n_bins),       # potential temperature
}
zBins_km_g = [
    (0, 3),
    
    (0, 1),
    (1, 3)]
zBins_km_c = [
    (0, 7),
    
    (1, 3),
    (3, 5),
    (5, 7)]

CL: ALL=1660231, SHALLOW=1220502, DEEP=42069
nonCL: ALL=712151, SHALLOW=551287, DEEP=12496
SBF: ALL=237335, SHALLOW=141001, DEEP=12635
ColdPool: ALL=1422896, SHALLOW=1079501, DEEP=29434
Mean Cloudbase is: 1.21 km

Min Cloudbase is: 1.18 km

Mean LFC is: 1.86 km

Mean LCL is: 1.77 km

Min LFC is: 1.25 km

Min LCL is: 1.23 km



In [ ]:
for t in tqdm(loop_elements, desc="Processing"):
    if t <= timesteps_per_hour:
        print(f"skipping time {t} since too close to first hour")
        continue
        
    print("#" * 40,"\n",f"Processing timestep {t}/{loop_elements[-1]}")
    timeString = ModelData.timeStrings[t]

    #Forming Dictionary for Profile Arrays for current timestep
    trackedProfileArrays = CopyStructure(trackedArrays)
    histogramsDictionary = InitializeHistograms(trackedProfileArrays,varNames, time_bins,zBins_km_g,zBins_km_c, property_bins_Dictionary)
    
    #getting variable data
    Z,Y,X = GetSpatialData(t)
    A_g,A_c = GetLangrangianBinaryArray(t)
    A_g_Prior,A_c_Prior = GetLangrangianBinaryArray(t-1)
    
    #making tracked profiles
    print("MAKING TRACKED PROFILES")
    histogramsDictionary = MakeTrackedProfiles(trackedArrays,histogramsDictionary,property_bins_Dictionary,varNames,
                                               Z,Y,X,t, A_g,A_c,A_g_Prior,A_c_Prior,
                                               zBins_km_g,zBins_km_c)
    
    #saving tracked profiles for current timestep
    TrackedProfiles_DataLoading_CLASS.SaveProfile(ModelData,DataManager_TrackedProfiles, histogramsDictionary, dataName="EntrainmentTrackback", t=t)

In [13]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE WHEN JOBARRAY IS RUNNING
# recombine=True

In [14]:
import copy
def RecombineProfiles(ModelData, DataManager):
    """
    Combine tracked profiles across all timesteps using the first as a template.
    """
    print(f"Recombining {ModelData.Ntime} TrackedProfiles files...\n")

    histogramsDictionary_combined = None

    for t in tqdm(range(ModelData.Ntime), desc="Combining Profiles", unit="timestep"):

        if t <= timesteps_per_hour:
            print(f"skipping time {t} since too close to first hour")
            continue
        
        histogramsDictionary = TrackedProfiles_DataLoading_CLASS.LoadProfile(ModelData, DataManager, dataName="EntrainmentTrackback", t=t)
         
        # --- initialize on first timestep ---
        if histogramsDictionary_combined is None:
            histogramsDictionary_combined = copy.deepcopy(histogramsDictionary)
            continue
    
        # --- accumulate later timesteps ---
        for key1 in histogramsDictionary:
            for key2 in histogramsDictionary[key1]:
                for varName in histogramsDictionary[key1][key2]:
                    for mode in histogramsDictionary[key1][key2][varName]:
                        for zKey in histogramsDictionary[key1][key2][varName][mode]:
                            histogramsDictionary_combined[key1][key2][varName][mode][zKey] += (
                                histogramsDictionary[key1][key2][varName][mode][zKey]
                            )
    return histogramsDictionary_combined


In [17]:
if recombine==True:

    histogramsDictionary_combined = RecombineProfiles(ModelData, DataManager_TrackedProfiles)
    TrackedProfiles_DataLoading_CLASS.SaveProfile(ModelData,DataManager_TrackedProfiles, 
                                                  histogramsDictionary_combined, dataName="EntrainmentTrackback", t='combined')

Recombining 661 TrackedProfiles files...



Combining Profiles:   0%|          | 0/661 [00:00<?, ?timestep/s]

skipping time 0 since too close to first hour
skipping time 1 since too close to first hour
skipping time 2 since too close to first hour
skipping time 3 since too close to first hour
skipping time 4 since too close to first hour
skipping time 5 since too close to first hour
skipping time 6 since too close to first hour
skipping time 7 since too close to first hour
skipping time 8 since too close to first hour
skipping time 9 since too close to first hour
skipping time 10 since too close to first hour
skipping time 11 since too close to first hour
skipping time 12 since too close to first hour
skipping time 13 since too close to first hour
skipping time 14 since too close to first hour
skipping time 15 since too close to first hour
skipping time 16 since too close to first hour
skipping time 17 since too close to first hour
skipping time 18 since too close to first hour
skipping time 19 since too close to first hour
skipping time 20 since too close to first hour
skipping time 21 since 

Combining Profiles: 100%|██████████| 661/661 [02:57<00:00,  3.72timestep/s]

Saved output to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Project_Algorithms/Tracked_Profiles/1km_1min_95nz/Tracked_Profiles/TrackedProfiles_EntrainmentTrackback_1km_1min_95nz_combined.pkl 



In [18]:
###################
#PLOTTING FUNCTIONS
plotting=False
plotting=True

In [19]:
#Loading Back In
if plotting:
    histogramsDictionary_combined = TrackedProfiles_DataLoading_CLASS.LoadProfile(ModelData,DataManager_TrackedProfiles, dataName="EntrainmentTrackback", t='combined')

In [20]:
def NormalizeHistogram(histogram):
    histogram_sum = histogram.sum(axis=1, keepdims=True)
    histogram_normalized = np.divide(histogram,histogram_sum, 
                          out=np.zeros_like(histogram, dtype=float),
                          where=histogram_sum != 0)
    return histogram_normalized

In [21]:
def CombinedPlot_PropertyHistogram_V1(parcelType,mode,zKey,
                                      plotType="contour",normalize = True):
    mins=ModelData.time[1].item()/1e9/60
    
    varNames     = ["QV", "QCQI", "W", "THETA_v"]
    parcelDepths = ["ALL", "SHALLOW", "DEEP"]

    nrows = len(parcelDepths)
    ncols = len(varNames)

    fig, axes = plt.subplots(
        nrows, ncols,
        figsize=(4.5 * ncols, 3.5 * nrows),
        constrained_layout=True
    )

    for i, parcelDepth in enumerate(parcelDepths):
        for j, varName in enumerate(varNames):

            ax = axes[i, j]

            a = histogramsDictionary_combined[parcelType][parcelDepth][varName][mode][zKey]

            if normalize:
                a = NormalizeHistogram(a)
                a *= 100
                colorbarTitle = "Frequency (%)" if j == ncols - 1 else ""
            else:
                colorbarTitle = "Count" if j == ncols - 1 else ""

            x = mins * time_bins
            y = property_bins_Dictionary[varName]

            x_centers = 0.5 * (x[:-1] + x[1:])
            y_centers = 0.5 * (y[:-1] + y[1:])
            X, Y = np.meshgrid(x_centers, y_centers)

            multiplier = 1e3 if varName in ["QV", "QCQI"] else 1

            if plotType == "contour":
                plotObject = ax.contourf(
                    X,
                    multiplier * Y,
                    a.T,
                    cmap="turbo",
                    levels=20
                )
            else:
                plotObject = ax.pcolormesh(
                    x,
                    multiplier * y,
                    a.T,
                    cmap="turbo",
                    shading="auto"
                )

            # ---- labels ----
            if i == nrows - 1:
                ax.set_xlabel("Backwards Time (mins)")
            if j == 0:
                ax.set_ylabel(f"{parcelDepth}\n{varName}")
            else:
                ax.set_ylabel(varName)

            ax.set_title(varName if i == 0 else "")

            plt.colorbar(plotObject, ax=ax, label=colorbarTitle)

    plt.suptitle(
        f"Parcel History Histograms ({parcelType}) ({zKey.replace('_', '-', 1).split("_")[0]})",
        fontsize=16
    )

    return fig

In [22]:
def GetPlottingDirectory(plotFileName, plotType):
    plottingDirectory = mainCodeDirectory=os.path.join(mainDirectory,"Code","PLOTTING")
    
    specificPlottingDirectory = os.path.join(plottingDirectory, plotType, 
                                             f"{ModelData.res}_{ModelData.t_res}_{ModelData.Nz_str}nz")
    os.makedirs(specificPlottingDirectory, exist_ok=True)

    plottingFileName=os.path.join(specificPlottingDirectory, plotFileName)

    return plottingFileName
    
def SaveFigure(fig,fileName,
               plotType=f"Project_Algorithms/Tracked_Profiles/Tracked_Profiles_EntrainmentTrackback"):
    plotFileName = f"{fileName}_{ModelData.res}_{ModelData.t_res}_{ModelData.Np_str}.jpg"
    plottingFileName = GetPlottingDirectory(plotFileName, plotType)
    
    print(f"Saving figure to {plottingFileName}")
    fig.savefig(plottingFileName, dpi=300, bbox_inches='tight')
    plt.close(fig) 

In [23]:
###################
#PLOTTING

In [ ]:
if plotting:
    parcelTypes = ["CL","nonCL","SBF"]
    for parcelType in tqdm(parcelTypes):
        for mode, zBins_km in (
                ("g", zBins_km_g),
                ("c", zBins_km_c)):
            
            for z1, z2 in zBins_km:
                zKey = f"{z1}_{z2}km_hist2d"
                fig = CombinedPlot_PropertyHistogram_V1(parcelType,mode,zKey)
                fileName = f"EntrainmentTrackback_PropertyHistogram_{parcelType}_{mode}_{zKey}"
                SaveFigure(fig,fileName)

In [ ]:
#####################
#DEPRECATED RECOMBINE (before adding mode and zKey)

In [ ]:
# import copy
# def RecombineProfiles(ModelData, DataManager):
#     """
#     Combine tracked profiles across all timesteps using the first as a template.
#     """
#     print(f"Recombining {ModelData.Ntime} TrackedProfiles files...\n")

#     histogramsDictionary_combined = None

#     for t in tqdm(range(ModelData.Ntime), desc="Combining Profiles", unit="timestep"):

#         if t <= timesteps_per_hour:
#             print(f"skipping time {t} since too close to first hour")
#             continue
        
#         histogramsDictionary = TrackedProfiles_DataLoading_CLASS.LoadProfile(ModelData, DataManager, dataName="EntrainmentTrackback", t=t)
         
#         # --- initialize on first timestep ---
#         if histogramsDictionary_combined is None:
#             histogramsDictionary_combined = copy.deepcopy(histogramsDictionary)
#             continue
    
#         # --- accumulate later timesteps ---
#         for key1 in histogramsDictionary:
#             for key2 in histogramsDictionary[key1]:
#                 for arrayName in histogramsDictionary[key1][key2]:
#                     histogramsDictionary_combined[key1][key2][arrayName] += (
#                         histogramsDictionary[key1][key2][arrayName]
#                     )
#     return histogramsDictionary_combined


In [ ]:
#####################
#DEPRECATED PLOT TYPE

In [ ]:
# def CombinedPlot_PropertyHistogram_V2(parcelType,
#                                      plotType="line",
#                                      normalize=True): #an idea for clearness, created a weighted mean plot #CURRENTLY DEPRECATED

#     mins = ModelData.time[1].item() / 1e9 / 60

#     varNames     = ["QV", "QCQI", "W", "THETA_v"]
#     parcelDepths = ["ALL", "SHALLOW", "DEEP"]

#     nrows = len(parcelDepths)
#     ncols = len(varNames)

#     fig, axes = plt.subplots(
#         nrows, ncols,
#         figsize=(4.5 * ncols, 3.5 * nrows),
#         constrained_layout=True
#     )

#     for i, parcelDepth in enumerate(parcelDepths):
#         for j, varName in enumerate(varNames):

#             ax = axes[i, j]

#             # ---- load histogram ----
#             # expected shape: (n_time_bins, n_property_bins)
#             a = histogramsDictionary_combined[parcelType][parcelDepth][
#                 f"{varName}_hist2d"
#             ]

#             if normalize:
#                 a = NormalizeHistogram(a)
#                 a *= 100.0

#             # ---- bins ----
#             x = mins * time_bins
#             y = property_bins_Dictionary[varName]

#             x_centers = 0.5 * (x[:-1] + x[1:])
#             y_centers = 0.5 * (y[:-1] + y[1:])

#             multiplier = 1e3 if varName in ["QV", "QCQI"] else 1.0

#             # ---- weighted mean over property bins ----
#             counts = a.T                     # (n_property_bins, n_time_bins)
#             total  = counts.sum(axis=0)      # per time bin

#             mean_y = np.full_like(total, np.nan, dtype=float)
#             valid  = total > 0

#             mean_y[valid] = (
#                 (counts[:, valid] * y_centers[:, None]).sum(axis=0)
#                 / total[valid]
#             )

#             # ---- plot ----
#             ax.plot(
#                 x_centers,
#                 multiplier * mean_y,
#                 lw=2,
#                 color="k"
#             )

#             # ---- labels ----
#             if i == nrows - 1:
#                 ax.set_xlabel("Backwards Time (mins)")
#             if j == 0:
#                 ax.set_ylabel(f"{parcelDepth}\n{varName}")
#             else:
#                 ax.set_ylabel(varName)

#             ax.set_title(varName if i == 0 else "")



    
#     #ALIGNING AXIS LIMITS
#     for j in range(ncols):
#         colAxes = list(axes[:, j])   
#         MatchAxisLimits_V2(colAxes, dim="y")
 
#     plt.suptitle(
#         f"Parcel History (Weighted Mean) ({parcelType})",
#         fontsize=16
#     )

#     return fig

# if plotting:
#     for parcelType in tqdm(parcelTypes):
#         fig = CombinedPlot_PropertyHistogram_V2(parcelType)
#         fileName = f"EntrainmentTrackback_PropertyHistogram_WeightedMean_{parcelType}"
#         SaveFigure(fig,fileName)

In [ ]:
###################
#DEPRECATED METHOD (LAST CLOUD TIME HISTOGRAM)

In [ ]:
# def InitializeHistograms(trackedArrays, varNames, time_bins, property_bins_Dictionary):
#     """
#     Create a nested structure matching trackedArrays,
#     with empty histogram arrays for each variable:
#         - var_hist2d
#         - var_parcel_last_time_hist2d
#     """
    
#     histogramsDictionary = {}
#     n_time = len(time_bins) - 1
    
#     for category, depth_dict in trackedArrays.items():  # e.g. 'CL', 'SBF'
#         histogramsDictionary[category] = {}

#         for depth_type in depth_dict.keys():  # e.g. 'ALL', 'SHALLOW', 'DEEP'
#             histogramsDictionary[category][depth_type] = {}

#             for varName in varNames:

#                 # number of property bins for this variable
#                 n_prop = len(property_bins_Dictionary[varName]) - 1

#                 # initialize empty histograms
#                 histogramsDictionary[category][depth_type][f"{varName}_hist2d"] = \
#                     np.zeros((n_time, n_prop))

#                 # histogramsDictionary[category][depth_type][f"{varName}_parcel_last_time_hist2d"] = \ #deprecated
#                 #     np.zeros((n_time, n_prop))
#     return histogramsDictionary

In [ ]:
# #FUNCTIONS FOR MAKING LAST CLOUD TIME HISTOGRAM

# def UpdateLastCloudState(last_cloud_time,last_property_value,
#                          VARs,collapsed,relative_time,count,
#                          qcqi_thresh=1e-6):
#     #GETTING LAST TIME PARCEL IN CLOUD
#     ##########
#     if count > 0: #make sure doesn't occur at 
#         # QCQI/QV for all entrained parcels
#         qcqi_values = VARs["QCQI"][collapsed]

#         # Which parcels currently in cloud
#         in_cloud_now = (qcqi_values >= qcqi_thresh)
        
#         # Which parcels have NOT been assigned a last-cloud-time yet
#         not_recorded = np.isnan(last_cloud_time)
#         # Newly identified parcels whose LAST cloud time is t_back
#         new_cloud_hits = in_cloud_now & not_recorded

#         # Update the last-cloud-time + variables at that time
#         last_cloud_time[new_cloud_hits] = relative_time
#         for varName, array in VARs.items():
#             property_values = array[collapsed]
#             last_property_value[varName][new_cloud_hits] = property_values[new_cloud_hits]
#     ##########


# def MakeLastCloudHistogram(histogramsDictionary,
#                            key1,key2,
#                            last_cloud_time,last_property_value,
#                            varNames,
#                            property_bins_Dictionary,time_bins):

#     # GETTING LAST TIME PARCEL IN CLOUD HISTOGRAM
#     ##########
#     valid = ~np.isnan(last_cloud_time)
#     if not np.any(valid):
#         return

#     for varName in varNames:
#         property_values = last_property_value[varName][valid]
#         property_bins   = property_bins_Dictionary[varName]
        
#         parcel_last_time_hist2d, _, _ = np.histogram2d(
#             last_cloud_time[valid],
#             property_values,
#             bins=(time_bins, property_bins)
#         )
#         histogramsDictionary[key1][key2][f"{varName}_parcel_last_time_hist2d"] = parcel_last_time_hist2d
#     ##########

In [ ]:
# def MakeTrackedProfiles(trackedArrays,histogramsDictionary,property_bins_Dictionary,varNames,Z,Y,X,t, A_g,A_c,A_g_Prior,A_c_Prior,
#                         printstatement=True):
#     """
#     Update profileArraysDictionary with variable data for parcels active at time t.
#     Accumulates sums and counts in both profile_array and profile_array_squares.
#     """
#     #CALCULATING
#     for key1, subdict in trackedArrays.items():         # e.g. 'CL', 'SBF'
#         print("\t",f'working on {key1}')
#         for key2, trackedArray in subdict.items():           # e.g. 'ALL', 'DEEP'
#             print("\t\t",f'working on {key2}')
    
#             #Part 1: getting parcels in trackedArray to run through
#             if printstatement: print(f"Part 1: getting parcels in trackedArray to run through")
                
#             _, selectedPs, leftRightIndexes = GetParcelNumbers(trackedArray, t) #get parcels that are counted at time t
#             if printstatement: print(f"\tRunning for {len(selectedPs)} Parcels")
            
#             #getting Z data
#             zLevels = Z[selectedPs]
#             yLevels = Y[selectedPs]
#             xLevels = X[selectedPs]

#             #Part 2: find which other parcels exist in each grid box
#             if printstatement: print(f"Part 2: find which other parcels exist in each grid box")
                
#             # Step a: compute spatial matches once
#             if printstatement: print("\tStep a: compute spatial matches once") #SLOW POINT HERE
#             gridboxMatches = GetGridBoxMatches_V2(Z,Y,X, zLevels,yLevels,xLevels)
#             if gridboxMatches is None:
#                 continue

#             #Part 3: find which of those parcels were entrained into a general/cloudy updraft
#             if printstatement: print(f"Part 3: find which of those parcels were entrained into a general/cloudy updraft")
            
#             # Step a: compute entrainment masks
#             if printstatement: print("\tStep a: compute entrainment masks")
#             mask_c = (A_c & (~A_c_Prior)).astype(bool)

#             # Step b: apply masks to find all parcels
#             if printstatement: print("\tStep b: apply masks to find all parcels") #SLOW POINT HERE
#             collapsed = GetWhereOtherEntrainedParcels_V2(mask_c,gridboxMatches)
#             if collapsed is None:
#                 continue

#             # Step c: track parcels back (last 30 minutes) and read properties
#             if printstatement: print("\tStep c: track parcels back (last 30 minutes) and read properties")
#             ##GETTING LAST TIME PARCEL IN CLOUD #deprecated
#             # last_cloud_time = np.full(len(collapsed), np.nan) #deprecated
#             # last_property_value = {varName: np.full(len(collapsed), np.nan) for varName in varNames} #deprecated
            
#             trackTimes = np.arange(t,(t-timesteps_per_hour)-1,-1)
#             for count, t_back in enumerate(tqdm(trackTimes,desc="\t\tTracking back parcels",leave=False)):
#                 relative_time = t_back - t

#                 VARs = MakeDataDictionary(varNames, t_back)   

#                 for varName, array in VARs.items():
#                     #GETTING PROPERTY HISTOGRAMS
#                     AccumulatePropertyHistogram(histogramsDictionary,
#                                                 key1,key2,varName,
#                                                 array,collapsed,relative_time,
#                                                 property_bins_Dictionary,time_bins)

#                 # # GETTING LAST TIME PARCEL IN CLOUD #***DEPRECATED***
#                 # UpdateLastCloudState(last_cloud_time,last_property_value,
#                 #                      VARs,collapsed,relative_time,count,
#                 #                      qcqi_thresh=1e-6)

#             # # GETTING LAST TIME PARCEL IN CLOUD HISTOGRAM #***DEPRECATED***
#             # MakeLastCloudHistogram(histogramsDictionary,
#             #                        key1,key2,
#             #                        last_cloud_time,last_property_value,
#             #                        varNames,
#             #                        property_bins_Dictionary,time_bins)
#     return histogramsDictionary

In [ ]:
# def CombinedPlot_LastCloudTimeHistogram_V1(parcelType,
#                                         plotType="contour",normalize=True):

#     mins=ModelData.time[1].item()/1e9/60

#     varNames     = ["QV", "QCQI", "W", "THETA_v"]
#     parcelDepths = ["ALL", "SHALLOW", "DEEP"]

#     nrows = len(parcelDepths)
#     ncols = len(varNames)

#     fig, axes = plt.subplots(
#         nrows, ncols,
#         figsize=(4.5 * ncols, 3.5 * nrows),
#         constrained_layout=True
#     )

#     for i, parcelDepth in enumerate(parcelDepths):
#         for j, varName in enumerate(varNames):

#             ax = axes[i, j]

#             a = histogramsDictionary_combined[parcelType][parcelDepth][
#                 f"{varName}_parcel_last_time_hist2d"
#             ]

#             if normalize:
#                 a = NormalizeHistogram(a)
#                 a *= 100
#                 colorbarTitle = "Frequency (%)" if j == ncols - 1 else ""
#             else:
#                 colorbarTitle = "Count" if j == ncols - 1 else ""

#             x = mins * time_bins
#             y = property_bins_Dictionary[varName]

#             x_centers = 0.5 * (x[:-1] + x[1:])
#             y_centers = 0.5 * (y[:-1] + y[1:])
#             X, Y = np.meshgrid(x_centers, y_centers)

#             multiplier = 1e3 if varName in ["QV", "QCQI"] else 1
#             colorbarLevels = 20 if varName not in ["QCQI"] else 20#np.linspace(0, 10)

#             if plotType == "contour":
#                 plotObject = ax.contourf(
#                     X,
#                     multiplier * Y,
#                     a.T,
#                     cmap="turbo",
#                     levels=colorbarLevels
#                 )
#             else:
#                 plotObject = ax.pcolormesh(
#                     x,
#                     multiplier * y,
#                     a.T,
#                     cmap="turbo",
#                     shading="auto"
#                 )

#             # ---- labels ----
#             if i == nrows - 1:
#                 ax.set_xlabel("Backwards Time (mins)")
#             if j == 0:
#                 ax.set_ylabel(f"{parcelDepth}\n{varName}")
#             else:
#                 ax.set_ylabel(varName)

#             ax.set_title(varName if i == 0 else "")

#             plt.colorbar(plotObject, ax=ax, label=colorbarTitle)

#     plt.suptitle(
#         f"Last-Cloud-Time Histograms ({parcelType})",
#         fontsize=16
#     )
#     return fig

# if plotting:
#     for parcelType in tqdm(parcelTypes):
#         fig = CombinedPlot_LastCloudTimeHistogram_V1(parcelType)
#         fileName = f"EntrainmentTrackback_LastCloudTimeHistogram_{parcelType}"
#         SaveFigure(fig,fileName)

In [ ]:
# def CombinedPlot_LastCloudTimeHistogram_V2(parcelType,
#                                            plotType="line",
#                                            normalize=True):

#     mins = ModelData.time[1].item() / 1e9 / 60

#     varNames     = ["QV", "QCQI", "W", "THETA_v"]
#     parcelDepths = ["ALL", "SHALLOW", "DEEP"]

#     nrows = len(parcelDepths)
#     ncols = len(varNames)

#     fig, axes = plt.subplots(
#         nrows, ncols,
#         figsize=(4.5 * ncols, 3.5 * nrows),
#         constrained_layout=True
#     )

#     for i, parcelDepth in enumerate(parcelDepths):
#         for j, varName in enumerate(varNames):

#             ax = axes[i, j]

#             # ---- load histogram ----
#             # shape after transpose: (n_property_bins, n_time_bins)
#             a = histogramsDictionary_combined[parcelType][parcelDepth][
#                 f"{varName}_parcel_last_time_hist2d"
#             ].T

#             # ---- bins ----
#             x = mins * time_bins
#             y = property_bins_Dictionary[varName]

#             x_centers = 0.5 * (x[:-1] + x[1:])
#             y_centers = 0.5 * (y[:-1] + y[1:])

#             multiplier = 1e3 if varName in ["QV", "QCQI"] else 1

#             # ---- weighted mean over property bins ----
#             counts = a
#             total  = counts.sum(axis=0)  # per time bin

#             mean_y = np.full_like(total, np.nan, dtype=float)
#             valid  = total > 0

#             mean_y[valid] = (
#                 (counts[:, valid] * y_centers[:, None]).sum(axis=0)
#                 / total[valid]
#             )

#             # ---- plot ----
#             ax.plot(
#                 x_centers,
#                 multiplier * mean_y,
#                 lw=2,
#                 color="k"
#             )

#             # ---- labels ----
#             if i == nrows - 1:
#                 ax.set_xlabel("Backwards Time (mins)")
#             if j == 0:
#                 ax.set_ylabel(f"{parcelDepth}\n{varName}")
#             else:
#                 ax.set_ylabel(varName)

#             ax.set_title(varName if i == 0 else "")


#     #ALIGNING AXIS LIMITS
#     for j in range(ncols):
#         colAxes = list(axes[:, j])   
#         MatchAxisLimits_V2(colAxes, dim="y")
        
#     plt.suptitle(
#         f"Last-Cloud-Time Property (Weighted Mean) ({parcelType})",
#         fontsize=16
#     )

#     return fig

# if plotting:
#     for parcelType in tqdm(parcelTypes):
#         fig = CombinedPlot_LastCloudTimeHistogram_V2(parcelType)
#         fileName = f"EntrainmentTrackback_LastCloudTimeHistogram_WeightedMean_{parcelType}"
#         SaveFigure(fig,fileName)